In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn
from keras import regularizers

In [4]:
from pathlib import Path
data_path_p_z_n = Path("/kaggle/input/hierarchical-dataset/model(+,-,neutral)_data")

train_dir_p_z_n = data_path_p_z_n / "train"
test_dir_p_z_n = data_path_p_z_n / "test"

In [5]:
data_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor() 
])

In [6]:
train_data_p_z_n = datasets.ImageFolder(root=train_dir_p_z_n,transform=data_transform,target_transform=None)
test_data_p_z_n = datasets.ImageFolder(root=test_dir_p_z_n,transform=data_transform)
print(f"Train data:\n{train_data_p_z_n}\nTest data:\n{test_data_p_z_n}")

Train data:
Dataset ImageFolder
    Number of datapoints: 28709
    Root location: /kaggle/input/hierarchical-dataset/model(+,-,neutral)_data/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 7178
    Root location: /kaggle/input/hierarchical-dataset/model(+,-,neutral)_data/test
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [103]:
class_names = train_data_p_z_n.classes
class_names
     

['middle', 'negative', 'positive']

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [108]:
train_dataloader_p_z_n = DataLoader(dataset=train_data_p_z_n,batch_size=32,shuffle=True)
test_dataloader_p_z_n = DataLoader(dataset=test_data_p_z_n,batch_size=32,shuffle=False)

In [7]:
class Emotionmodel(nn.Module):
  def __init__(self,input_channels:int,out_channel:int):
    super().__init__()
    self.layer_stack1 = nn.Sequential(
      nn.Conv2d(in_channels=input_channels,out_channels=32, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=32,out_channels=64, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack2 = nn.Sequential(
      nn.Conv2d(in_channels=64,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Conv2d(in_channels=128,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack3 = nn.Sequential(
      nn.Flatten(),
      # 6x6= 36
      #we checked the in features by multiplication in between them.
      nn.Linear(in_features =36*128,out_features=256),
      nn.ReLU(),
      nn.Dropout(p=0.5),
      nn.Linear(in_features =256 ,out_features=3),
      nn.Softmax(dim=1)
    )
  def forward(self,x):
    x = self.layer_stack1(x)
    #print(x.shape)
    #x = x.unsqueeze(dim=0)
    #print(x.shape)
    x = self.layer_stack2(x)
    #print(x.shape)
    #f = nn.Flatten()
    #o=f(x)
    #print(o.shape)
    x = self.layer_stack3(x)
    #print(x.shape)
    return x
     

In [10]:
torch.manual_seed(42)
model_p_z_n = Emotionmodel(1,out_channel = len(train_data_p_z_n.classes)).to(device)
model_p_z_n

Emotionmodel(
  (layer_stack1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
  )
  (layer_stack2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout(p=0.25, inplace=False)
  )
  (layer_stack3): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4608, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=25

In [107]:
img=0
for X,y in train_dataloader:
  img = X[0,:,:,:]
  break
print(img.shape)

NameError: name 'train_dataloader' is not defined

In [51]:
test_img1 = img.unsqueeze(dim=1).to(device)
test_img.shape

torch.Size([1, 1, 48, 48])

In [18]:
model_p_z_n(test_img)
     

tensor([[0.3212, 0.3340, 0.3448]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [109]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_p_z_n.parameters(),lr=0.0001,weight_decay = 1e-6)

In [110]:

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


In [111]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
def train_time(start: float,end: float,device:torch.device = None):
  tot_time = end - start
  print(f"time taken:{tot_time:.3f} seconds , on device:{device}")
  return tot_time


In [112]:
def train_step(model:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device=device):
  train_loss,train_acc =0,0
  #model.train()
  for batch_number ,(X,y) in enumerate(data_loader):
    X,y = X.to(device),y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred,y)
    # loss for every batches  
    train_loss+=loss
    train_acc +=accuracy_fn(y,y_pred.argmax(dim=1))  
    #optimitzer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

In [113]:
def test_step(model:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:nn.Module,
               accuracy_fn,
               device:torch.device=device):
  model.eval()
  test_loss,test_acc=0,0
  with torch.inference_mode():
    for X,y in data_loader:
      X,y = X.to(device),y.to(device)
      test_pred = model(X)
      test_loss+=loss_fn(test_pred,y)
      test_acc+=accuracy_fn(y,test_pred.argmax(dim=1))
    test_loss/=len(data_loader)
    test_acc/=len(data_loader)
    print(f"\nTest loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n") 


In [114]:
def eval_model(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               accuracy_fn):
   #return dictionary of results
  loss,acc=0,0
  with torch.inference_mode():
    for X,y in data_loader:
      X,y = X.to(device),y.to(device)
      eval_pred = model(X)
      loss +=loss_fn(eval_pred,y) 
      acc  +=accuracy_fn(y,eval_pred.argmax(dim=1))
    loss /=len(data_loader)
    acc /=len(data_loader)
  return {"model_name":model.__class__.__name__,
          "model_loss":loss.item(),
          "model_acc":acc}   

In [115]:
torch.manual_seed(42)
start_time=timer()
epochs = 30
for epoch in tqdm(range(epochs)):
   print(f"Epoch: {epoch}\n---------")
   train_step(model_p_z_n,train_dataloader_p_z_n,loss_fn,optimizer,accuracy_fn,device)
   test_step(model_p_z_n,test_dataloader_p_z_n,loss_fn,accuracy_fn,device)
end_time = timer()
train_time(start_time,end_time,device)   

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 1.07898 | Train accuracy: 40.98%

Test loss: 1.06611, Test acc: 44.59%

Epoch: 1
---------
Train loss: 1.04188 | Train accuracy: 47.05%

Test loss: 1.00993, Test acc: 51.64%

Epoch: 2
---------
Train loss: 0.99876 | Train accuracy: 52.79%

Test loss: 0.98747, Test acc: 54.37%

Epoch: 3
---------
Train loss: 0.97709 | Train accuracy: 55.13%

Test loss: 0.97239, Test acc: 54.94%

Epoch: 4
---------
Train loss: 0.96111 | Train accuracy: 56.94%

Test loss: 0.95578, Test acc: 57.50%

Epoch: 5
---------
Train loss: 0.94549 | Train accuracy: 58.71%

Test loss: 0.96053, Test acc: 56.54%

Epoch: 6
---------
Train loss: 0.93326 | Train accuracy: 60.00%

Test loss: 0.93617, Test acc: 59.24%

Epoch: 7
---------
Train loss: 0.91907 | Train accuracy: 61.54%

Test loss: 0.92306, Test acc: 61.12%

Epoch: 8
---------
Train loss: 0.90678 | Train accuracy: 62.97%

Test loss: 0.92106, Test acc: 61.30%

Epoch: 9
---------
Train loss: 0.89643 | Train accuracy: 64.15%

Test los

1115.9541115909997

In [32]:
model_conv_results = eval_model(model_p_z_n,test_dataloader,loss_fn,accuracy_fn)
model_conv_results

{'model_name': 'Emotionmodel',
 'model_loss': 0.8566949963569641,
 'model_acc': 68.35833333333333}

# Angry_fear


In [3]:

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn
from keras import regularizers

In [11]:
from pathlib import Path
data_path_angry_fear = Path("/kaggle/input/hierarchical-dataset/model_angry_fear_negative_data")

train_dir_angry_fear = data_path_angry_fear / "train"
test_dir_angry_fear = data_path_angry_fear / "test"

In [4]:
data_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor() 
])

In [12]:
train_data_angry_fear = datasets.ImageFolder(root=train_dir_angry_fear,transform=data_transform,target_transform=None)
test_data_angry_fear = datasets.ImageFolder(root=test_dir_angry_fear,transform=data_transform)
print(f"Train data:\n{train_data_angry_fear}\nTest data:\n{test_data_angry_fear}")

Train data:
Dataset ImageFolder
    Number of datapoints: 8092
    Root location: /kaggle/input/hierarchical-dataset/model_angry_fear_negative_data/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 1982
    Root location: /kaggle/input/hierarchical-dataset/model_angry_fear_negative_data/test
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [13]:
train_dataloader_angry_fear = DataLoader(dataset=train_data_angry_fear,batch_size=32,shuffle=True)
test_dataloader_angry_fear = DataLoader(dataset=test_data_angry_fear,batch_size=32,shuffle=False)

In [8]:
class_names = train_data_angry_fear.classes
class_names

['angry', 'fear']

In [37]:
class Emotionmodel_angry_fear(nn.Module):
  def __init__(self,input_channels:int,out_channel:int):
    super().__init__()
    self.layer_stack1 = nn.Sequential(
      nn.Conv2d(in_channels=input_channels,out_channels=32, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=32,out_channels=64, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack2 = nn.Sequential(
      nn.Conv2d(in_channels=64,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Conv2d(in_channels=128,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack3 = nn.Sequential(
      nn.Flatten(),
      # 6x6= 36
      #we checked the in features by multiplication in between them.
      nn.Linear(in_features =36*128,out_features=256),
      nn.ReLU(),
      nn.Dropout(p=0.5),
      nn.Linear(in_features =256 ,out_features=2),
      nn.Softmax(dim=1)
    )
  def forward(self,x):
    x = self.layer_stack1(x)
    #print(x.shape)
    #x = x.unsqueeze(dim=0)
    #print(x.shape)
    x = self.layer_stack2(x)
    #print(x.shape)
    #f = nn.Flatten()
    #o=f(x)
    #print(o.shape)
    x = self.layer_stack3(x)
    #print(x.shape)
    return x

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [38]:
torch.manual_seed(42)

model_angry_fear = Emotionmodel_angry_fear(1,out_channel = len(train_data_angry_fear.classes)).to(device)
model_angry_fear

Emotionmodel_angry_fear(
  (layer_stack1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
  )
  (layer_stack2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout(p=0.25, inplace=False)
  )
  (layer_stack3): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4608, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_

In [87]:
img=0
for X,y in train_dataloader_angry_fear:
  img = X[0,:,:,:]
  break
test_img = img.unsqueeze(dim=1).to(device)
test_img.shape

torch.Size([1, 1, 48, 48])

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_angry_fear.parameters(),lr=0.0001,weight_decay = 1e-6)

In [88]:
model_angry_fear(test_img)

tensor([[0.9825, 0.0175]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [15]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


In [16]:
def train_step(model:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device=device):
  train_loss,train_acc =0,0
  #model.train()
  for batch_number ,(X,y) in enumerate(data_loader):
    X,y = X.to(device),y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred,y)
    # loss for every batches  
    train_loss+=loss
    train_acc +=accuracy_fn(y,y_pred.argmax(dim=1))  
    #optimitzer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

In [17]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
def train_time(start: float,end: float,device:torch.device = None):
  tot_time = end - start
  print(f"time taken:{tot_time:.3f} seconds , on device:{device}")
  return tot_time

In [18]:
def test_step(model:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:nn.Module,
               accuracy_fn,
               device:torch.device=device):
  model.eval()
  test_loss,test_acc=0,0
  with torch.inference_mode():
    for X,y in data_loader:
      X,y = X.to(device),y.to(device)
      test_pred = model(X)
      test_loss+=loss_fn(test_pred,y)
      test_acc+=accuracy_fn(y,test_pred.argmax(dim=1))
    test_loss/=len(data_loader)
    test_acc/=len(data_loader)
    print(f"\nTest loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n") 

In [41]:
def eval_model(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               accuracy_fn):
   #return dictionary of results
  loss,acc=0,0
  with torch.inference_mode():
    for X,y in data_loader:
      X,y = X.to(device),y.to(device)
      eval_pred = model(X)
      loss +=loss_fn(eval_pred,y) 
      acc  +=accuracy_fn(y,eval_pred.argmax(dim=1))
    loss /=len(data_loader)
    acc /=len(data_loader)
  return {"model_name":model.__class__.__name__,
          "model_loss":loss.item(),
          "model_acc":acc}   

In [21]:
torch.manual_seed(42)
start_time=timer()
epochs = 30
for epoch in tqdm(range(epochs)):
   print(f"Epoch: {epoch}\n---------")
   train_step(model_angry_fear,train_dataloader_angry_fear,loss_fn,optimizer,accuracy_fn,device)
   test_step(model_angry_fear,test_dataloader_angry_fear,loss_fn,accuracy_fn,device)
end_time = timer()
train_time(start_time,end_time,device)
torch.save(model_angry_fear, 'model_angry_fear.pth')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.39702 | Train accuracy: 91.80%

Test loss: 0.58468, Test acc: 72.30%

Epoch: 1
---------
Train loss: 0.39766 | Train accuracy: 91.83%

Test loss: 0.57743, Test acc: 73.01%

Epoch: 2
---------
Train loss: 0.39776 | Train accuracy: 91.72%

Test loss: 0.57583, Test acc: 73.02%

Epoch: 3
---------
Train loss: 0.39860 | Train accuracy: 91.68%

Test loss: 0.59315, Test acc: 71.28%

Epoch: 4
---------
Train loss: 0.39875 | Train accuracy: 91.58%

Test loss: 0.57922, Test acc: 72.91%

Epoch: 5
---------
Train loss: 0.39298 | Train accuracy: 92.20%

Test loss: 0.59186, Test acc: 71.24%

Epoch: 6
---------
Train loss: 0.39520 | Train accuracy: 92.04%

Test loss: 0.58369, Test acc: 71.85%

Epoch: 7
---------
Train loss: 0.38948 | Train accuracy: 92.56%

Test loss: 0.59087, Test acc: 71.44%

Epoch: 8
---------
Train loss: 0.39415 | Train accuracy: 92.19%

Test loss: 0.58350, Test acc: 72.10%

Epoch: 9
---------
Train loss: 0.39689 | Train accuracy: 91.88%

Test los

In [ ]:
model_angry_fear_results = eval_model(model_angry_fear,test_dataloader_angry_fear,loss_fn,accuracy_fn)
model_angry_fear_results

# Disgust_neutral_sad

In [14]:
from pathlib import Path
data_path_disgust_neutral_sad = Path("/kaggle/input/hierarchical-dataset/model_disgust_neutral_sad_middle_data")

train_dir_disgust_neutral_sad = data_path_disgust_neutral_sad / "train"
test_dir_disgust_neutral_sad = data_path_disgust_neutral_sad / "test"

In [15]:
train_data_disgust_neutral_sad = datasets.ImageFolder(root=train_dir_disgust_neutral_sad,transform=data_transform,target_transform=None)
test_data_disgust_neutral_sad = datasets.ImageFolder(root=test_dir_disgust_neutral_sad,transform=data_transform)
print(f"Train data:\n{train_data_disgust_neutral_sad}\nTest data:\n{test_data_disgust_neutral_sad}")

Train data:
Dataset ImageFolder
    Number of datapoints: 10231
    Root location: /kaggle/input/hierarchical-dataset/model_disgust_neutral_sad_middle_data/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 2591
    Root location: /kaggle/input/hierarchical-dataset/model_disgust_neutral_sad_middle_data/test
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [17]:
train_dataloader_disgust_neutral_sad = DataLoader(dataset=train_data_disgust_neutral_sad,batch_size=32,shuffle=True)
test_dataloader_disgust_neutral_sad = DataLoader(dataset=test_data_disgust_neutral_sad,batch_size=32,shuffle=False)

In [42]:
class_names = train_data_disgust_neutral_sad.classes
class_names

['disgust', 'neutral', 'sad']

In [35]:
class Emotionmodel_disgust_neutral_sad(nn.Module):
  def __init__(self,input_channels:int,out_channel:int):
    super().__init__()
    self.layer_stack1 = nn.Sequential(
      nn.Conv2d(in_channels=input_channels,out_channels=32, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=32,out_channels=64, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack2 = nn.Sequential(
      nn.Conv2d(in_channels=64,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Conv2d(in_channels=128,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack3 = nn.Sequential(
      nn.Flatten(),
      # 6x6= 36
      #we checked the in features by multiplication in between them.
      nn.Linear(in_features =36*128,out_features=256),
      nn.ReLU(),
      nn.Dropout(p=0.5),
      nn.Linear(in_features =256 ,out_features=3),
      nn.Softmax(dim=1)
    )
  def forward(self,x):
    x = self.layer_stack1(x)
    #print(x.shape)
    #x = x.unsqueeze(dim=0)
    #print(x.shape)
    x = self.layer_stack2(x)
    #print(x.shape)
    #f = nn.Flatten()
    #o=f(x)
    #print(o.shape)
    x = self.layer_stack3(x)
    #print(x.shape)
    return x

In [36]:
torch.manual_seed(42)
model_disgust_neutral_sad = Emotionmodel_disgust_neutral_sad(1,out_channel = len(train_data_disgust_neutral_sad.classes)).to(device)
model_disgust_neutral_sad

Emotionmodel_disgust_neutral_sad(
  (layer_stack1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
  )
  (layer_stack2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout(p=0.25, inplace=False)
  )
  (layer_stack3): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4608, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): L

In [46]:
img=0
for X,y in train_dataloader_disgust_neutral_sad:
  img = X[0,:,:,:]
  break
test_img = img.unsqueeze(dim=1).to(device)
test_img.shape

torch.Size([1, 1, 48, 48])

In [47]:
model_disgust_neutral_sad(test_img)

tensor([[0.3212, 0.3396, 0.3391]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [49]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_disgust_neutral_sad.parameters(),lr=0.0001,weight_decay = 1e-6)

In [50]:
torch.manual_seed(42)
start_time=timer()
epochs = 35
for epoch in tqdm(range(epochs)):
   print(f"Epoch: {epoch}\n---------")
   train_step(model_disgust_neutral_sad,train_dataloader_disgust_neutral_sad,loss_fn,optimizer,accuracy_fn,device)
   test_step(model_disgust_neutral_sad,test_dataloader_disgust_neutral_sad,loss_fn,accuracy_fn,device)
end_time = timer()
train_time(start_time,end_time,device)
torch.save(model_disgust_neutral_sad, 'model_disgust_neutral_sad.pth')

  0%|          | 0/35 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.98575 | Train accuracy: 48.81%

Test loss: 0.97518, Test acc: 49.93%

Epoch: 1
---------
Train loss: 0.95686 | Train accuracy: 55.31%

Test loss: 0.93947, Test acc: 58.89%

Epoch: 2
---------
Train loss: 0.93837 | Train accuracy: 58.33%

Test loss: 0.93372, Test acc: 59.06%

Epoch: 3
---------
Train loss: 0.92625 | Train accuracy: 60.36%

Test loss: 0.93285, Test acc: 58.64%

Epoch: 4
---------
Train loss: 0.91868 | Train accuracy: 61.54%

Test loss: 0.91217, Test acc: 62.33%

Epoch: 5
---------
Train loss: 0.91372 | Train accuracy: 62.00%

Test loss: 0.90762, Test acc: 62.44%

Epoch: 6
---------
Train loss: 0.90792 | Train accuracy: 62.65%

Test loss: 0.90653, Test acc: 62.56%

Epoch: 7
---------
Train loss: 0.90453 | Train accuracy: 63.07%

Test loss: 0.90637, Test acc: 62.71%

Epoch: 8
---------
Train loss: 0.90103 | Train accuracy: 63.49%

Test loss: 0.90127, Test acc: 62.84%

Epoch: 9
---------
Train loss: 0.89626 | Train accuracy: 64.06%

Test los

In [51]:
model_disgust_neutral_sad_results = eval_model(model_disgust_neutral_sad,test_dataloader_disgust_neutral_sad,loss_fn,accuracy_fn)
model_disgust_neutral_sad_results

{'model_name': 'Emotionmodel_disgust_neutral_sad',
 'model_loss': 0.8673153519630432,
 'model_acc': 67.12091796097172}

# surprise_happy

In [19]:
from pathlib import Path
data_path_suprise_happy = Path("/kaggle/input/hierarchical-dataset/model_suprise_happy_positive_data")

train_dir_suprise_happy = data_path_suprise_happy / "train"
test_dir_suprise_happy = data_path_suprise_happy/ "test"

In [20]:
train_data_suprise_happy = datasets.ImageFolder(root=train_dir_suprise_happy,transform=data_transform,target_transform=None)
test_data_suprise_happy = datasets.ImageFolder(root=test_dir_suprise_happy,transform=data_transform)
print(f"Train data:\n{train_data_suprise_happy}\nTest data:\n{test_data_suprise_happy}")

Train data:
Dataset ImageFolder
    Number of datapoints: 10386
    Root location: /kaggle/input/hierarchical-dataset/model_suprise_happy_positive_data/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 2605
    Root location: /kaggle/input/hierarchical-dataset/model_suprise_happy_positive_data/test
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [21]:
train_dataloader_suprise_happy = DataLoader(dataset=train_data_suprise_happy,batch_size=32,shuffle=True)
test_dataloader_suprise_happy = DataLoader(dataset=test_data_suprise_happy,batch_size=32,shuffle=False)

In [57]:
class_names = train_data_suprise_happy.classes
class_names

['happy', 'surprise']

In [33]:
class Emotionmodel_suprise_happy(nn.Module):
  def __init__(self,input_channels:int,out_channel:int):
    super().__init__()
    self.layer_stack1 = nn.Sequential(
      nn.Conv2d(in_channels=input_channels,out_channels=32, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=32,out_channels=64, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack2 = nn.Sequential(
      nn.Conv2d(in_channels=64,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Conv2d(in_channels=128,out_channels=128, kernel_size = 3, padding =1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(p=0.25),
    )
    self.layer_stack3 = nn.Sequential(
      nn.Flatten(),
      # 6x6= 36
      #we checked the in features by multiplication in between them.
      nn.Linear(in_features =36*128,out_features=256),
      nn.ReLU(),
      nn.Dropout(p=0.5),
      nn.Linear(in_features =256 ,out_features=2),
      nn.Softmax(dim=1)
    )
  def forward(self,x):
    x = self.layer_stack1(x)
    #print(x.shape)
    #x = x.unsqueeze(dim=0)
    #print(x.shape)
    x = self.layer_stack2(x)
    #print(x.shape)
    #f = nn.Flatten()
    #o=f(x)
    #print(o.shape)
    x = self.layer_stack3(x)
    #print(x.shape)
    return x

In [34]:
torch.manual_seed(42)

model_suprise_happy = Emotionmodel_suprise_happy(1,out_channel = len(train_data_suprise_happy.classes)).to(device)
model_suprise_happy

Emotionmodel_suprise_happy(
  (layer_stack1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
  )
  (layer_stack2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout(p=0.25, inplace=False)
  )
  (layer_stack3): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4608, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(

In [60]:
img=0
for X,y in train_dataloader_suprise_happy:
  img = X[0,:,:,:]
  break
test_img = img.unsqueeze(dim=1).to(device)
test_img.shape

torch.Size([1, 1, 48, 48])

In [61]:
model_suprise_happy(test_img)

tensor([[0.4883, 0.5117]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [63]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_suprise_happy.parameters(),lr=0.0001,weight_decay = 1e-6)

In [65]:
torch.manual_seed(42)
start_time=timer()
epochs = 70
for epoch in tqdm(range(epochs)):
   print(f"Epoch: {epoch}\n---------")
   train_step(model_suprise_happy,train_dataloader_suprise_happy,loss_fn,optimizer,accuracy_fn,device)
   test_step(model_suprise_happy,test_dataloader_suprise_happy,loss_fn,accuracy_fn,device)
end_time = timer()
train_time(start_time,end_time,device)
torch.save(model_suprise_happy, 'model_suprise_happy.pth')

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.60664 | Train accuracy: 70.35%

Test loss: 0.55237, Test acc: 75.33%

Epoch: 1
---------
Train loss: 0.50126 | Train accuracy: 80.80%

Test loss: 0.47064, Test acc: 83.22%

Epoch: 2
---------
Train loss: 0.45832 | Train accuracy: 85.18%

Test loss: 0.44968, Test acc: 85.48%

Epoch: 3
---------
Train loss: 0.44564 | Train accuracy: 86.32%

Test loss: 0.44201, Test acc: 86.47%

Epoch: 4
---------
Train loss: 0.44065 | Train accuracy: 86.83%

Test loss: 0.43450, Test acc: 87.20%

Epoch: 5
---------
Train loss: 0.43251 | Train accuracy: 87.69%

Test loss: 0.43981, Test acc: 86.91%

Epoch: 6
---------
Train loss: 0.42887 | Train accuracy: 88.05%

Test loss: 0.42737, Test acc: 87.69%

Epoch: 7
---------
Train loss: 0.42277 | Train accuracy: 88.78%

Test loss: 0.42152, Test acc: 88.46%

Epoch: 8
---------
Train loss: 0.41928 | Train accuracy: 88.97%

Test loss: 0.41754, Test acc: 89.22%

Epoch: 9
---------
Train loss: 0.41633 | Train accuracy: 89.33%

Test los

In [66]:
model_suprise_happy_results = eval_model(model_suprise_happy,test_dataloader_suprise_happy,loss_fn,accuracy_fn)
model_suprise_happy_results

{'model_name': 'Emotionmodel_suprise_happy',
 'model_loss': 0.3879503607749939,
 'model_acc': 92.33993902439025}

# Training Part Done!!
# Now, Creating Custom Testing Loop

In [104]:
model_p_z_n = torch.load('/kaggle/input/model-weights-pytorch/model_weights/model_p_z_n.pth')
model_disgust_neutral_sad = torch.load('/kaggle/input/model-weights-pytorch/model_weights/model_disgust_neutral_sad.pth')
model_angry_fear = torch.load('/kaggle/input/model-weights-pytorch/model_weights/model_angry_fear (1).pth')
model_suprise_happy = torch.load('/kaggle/input/model-weights-pytorch/model_weights/model_suprise_happy.pth')

In [128]:
model_p_z_n_results = eval_model(model_p_z_n,test_dataloader_p_z_n,loss_fn,accuracy_fn)
model_p_z_n_results

{'model_name': 'Emotionmodel',
 'model_loss': 0.8592656254768372,
 'model_acc': 67.96944444444445}

In [127]:
model_disgust_neutral_sad_results = eval_model(model_disgust_neutral_sad,test_dataloader_disgust_neutral_sad,loss_fn,accuracy_fn)
model_disgust_neutral_sad_results

{'model_name': 'Emotionmodel_disgust_neutral_sad',
 'model_loss': 0.870993435382843,
 'model_acc': 66.84961170848267}

In [125]:
model_angry_fear_results = eval_model(model_angry_fear,test_dataloader_angry_fear,loss_fn,accuracy_fn)
model_angry_fear_results

{'model_name': 'Emotionmodel_angry_fear',
 'model_loss': 0.5783975720405579,
 'model_acc': 72.81586021505376}

In [129]:
model_suprise_happy_results = eval_model(model_suprise_happy,test_dataloader_suprise_happy,loss_fn,accuracy_fn)
model_suprise_happy_results

{'model_name': 'Emotionmodel_suprise_happy',
 'model_loss': 0.38752490282058716,
 'model_acc': 92.45426829268293}

In [43]:
loss_fn = nn.CrossEntropyLoss()
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


In [68]:
class_names = train_data_suprise_happy.classes
class_names

['happy', 'surprise']

In [70]:
class_names_all= test_data_all.classes
class_names_all

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [123]:
def test_loop(model_p_z_n:nn.Module,
              model_angry_fear:nn.Module,
              model_disgust_neutral_sad:nn.Module,
              model_surprise_happy:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               device:torch.device=device):
  model_surprise_happy.eval()
  model_angry_fear.eval()
  model_disgust_neutral_sad.eval()
  model_p_z_n.eval()
  count=0
  with torch.inference_mode():
    for X,y in data_loader:
      X,y = X.to(device),y.to(device)
      test_pred_middle= model_p_z_n(X) 
      indices = torch.argmax(test_pred_middle).item()
#       print(indices)
      if indices == 0:
        test_pred = model_disgust_neutral_sad(X)
        class_names = train_data_disgust_neutral_sad.classes
      elif indices == 1:
        test_pred = model_angry_fear(X)
        class_names = train_data_angry_fear.classes
      else :
        test_pred = model_surprise_happy(X)
        class_names = train_data_suprise_happy.classes
      final_indices = torch.argmax(test_pred).item()
      predicted_emotion = class_names[final_indices]
      final_label = torch.argmax(y)
      label_emotion = class_names_all[final_label]
      if predicted_emotion == label_emotion:
        count+=1
    accuracy_final = count/len(data_loader)
    print(accuracy_final)

In [54]:
from pathlib import Path
data_path_all = Path("/kaggle/input/fer2013")

test_dir_all = data_path_all / "test"

In [60]:
test_data_all = datasets.ImageFolder(root=test_dir_all,transform=data_transform)
print(f"Test data:\n{test_data_all}")

Test data:
Dataset ImageFolder
    Number of datapoints: 7178
    Root location: /kaggle/input/fer2013/test
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [89]:
test_dataloader_all = DataLoader(dataset=test_data_all,batch_size=1,shuffle=False)

In [119]:
len(test_dataloader_all)

7178

In [ ]:
test_loop(model_p_z_n,model_angry_fear,model_disgust_neutral_sad,model_suprise_happy,test_dataloader_all,device)